In [1]:
!pip install -q ragas datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.3/320.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import os
from datasets import load_dataset, Dataset
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
from ragas import evaluate
import json
from langchain_openai import ChatOpenAI

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [4]:
# Set OPENAI_API_KEY

In [5]:
# Define root_path and data_path

In [ ]:
# Vector

# Prepare to accumulate DataFrames
dfs = []

# Loop through the files in the specified directory
for i in range(1, 11):  # Assuming files are numbered from 1 to 10
    file_name = f'file_name'
    file_path = os.path.join(data_path, file_name)

    # Check if the file exists
    if os.path.exists(file_path):
        # Load the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Append the DataFrame to the list
        dfs.append(df)
    else:
        print(f'File not found: {file_name}')

# Concatenate all DataFrames in the list into a single DataFrame
vector_df = pd.concat(dfs, ignore_index=True)

print(vector_df.shape)

In [ ]:
# Graph

# List to accumulate DataFrames
dfs = []

# Loop through the specified range, including the special case
for i in range(1, 6):  # Files are numbered from 1 to 5
    if i != 4:
        # For standard files
        file_name = f'file_name'
        file_path = os.path.join(data_path, file_name)
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            dfs.append(df)
        else:
            print(f'File not found: {file_name}')
    else:
        # For the special case with files 4_1 to 4_3
        for j in range(1, 4):  # Sub-files are numbered from 1 to 3
            file_name = f'file_name'
            file_path = os.path.join(data_path, file_name)
            if os.path.exists(file_path):
                df = pd.read_csv(file_path)
                dfs.append(df)
            else:
                print(f'File not found: {file_name}')

# Concatenate all DataFrames in the list into a single DataFrame
graph_df = pd.concat(dfs, ignore_index=True)

print(graph_df.shape)

In [ ]:
vector_df.dropna(subset=['teacher_answer'], inplace=True)
graph_df.dropna(subset=['teacher_answer'], inplace=True)
print(vector_df.shape)
print(graph_df.shape)

In [9]:
# Load saved_df
sampled_ids = saved_df['id'].tolist()

In [ ]:
sampled_vector_df = vector_df[vector_df['id'].isin(sampled_ids)]
sampled_graph_df = graph_df[graph_df['id'].isin(sampled_ids)]

print(sampled_vector_df.shape)
print(sampled_graph_df.shape)

In [11]:
sub_df_vector = sampled_vector_df.drop('id', axis=1)
sub_df_graph = sampled_graph_df.drop('id', axis=1)

In [12]:
def filter_columns(df, mode):
    """
    Removes columns based on the mode ('no', 'low', or 'high').

    Parameters:
    - df: Pandas DataFrame to process.
    - mode: String specifying the mode. Accepts 'no', 'low', or 'high'.

    Returns:
    - A new DataFrame with the specified columns removed.
    """

    # Ensure mode is either 'no', 'low', or 'high'
    if mode not in ['no', 'low', 'high']:
        raise ValueError("Mode must be 'no', 'low', or 'high'")

    # Determine the substrings to keep based on the mode
    if mode == 'no':
        filters_to_keep = ['no']
    elif mode == 'low':
        filters_to_keep = ['low']
    elif mode == 'high':
        filters_to_keep = ['high']

    # Identify columns that match the filters to keep
    columns_to_keep = ['question', 'teacher_answer'] + [col for col in df.columns if any(f in col for f in filters_to_keep)]

    # Drop columns not in the columns_to_keep list and return the new DataFrame
    return df[columns_to_keep]

In [13]:
sub_df_vector_no = filter_columns(sub_df_vector, 'no')
sub_df_vector_low = filter_columns(sub_df_vector, 'low')
sub_df_vector_high = filter_columns(sub_df_vector, 'high')
sub_df_graph_no = filter_columns(sub_df_graph, 'no')
sub_df_graph_low = filter_columns(sub_df_graph, 'low')
sub_df_graph_high = filter_columns(sub_df_graph, 'high')

In [14]:
def rename_columns(df):
    """
    Renames columns in the DataFrame as specified:
    - 'teacher_answer' to 'ground_truth',
    - Columns containing 'response' to include 'answer',
    - Columns containing 'context' to include 'contexts'.

    Parameters:
    - df: Pandas DataFrame to process.

    Returns:
    - A new DataFrame with columns renamed.
    """
    # Create a copy of the DataFrame to avoid modifying the original
    new_df = df.copy()

    # Rename 'teacher_answer' to 'ground_truth'
    new_df.rename(columns={'teacher_answer': 'ground_truth'}, inplace=True)

    # Rename columns containing 'response' and 'context'
    new_df.columns = ['answer' if 'response' in col else col for col in new_df.columns]
    new_df.columns = ['contexts' if 'context' in col else col for col in new_df.columns]

    return new_df

In [15]:
sub_df_vector_no_renamed = rename_columns(sub_df_vector_no)
sub_df_vector_low_renamed = rename_columns(sub_df_vector_low)
sub_df_vector_high_renamed = rename_columns(sub_df_vector_high)
sub_df_graph_no_renamed = rename_columns(sub_df_graph_no)
sub_df_graph_low_renamed = rename_columns(sub_df_graph_low)
sub_df_graph_high_renamed = rename_columns(sub_df_graph_high)

In [16]:
print(sub_df_vector_no_renamed.columns)
print(sub_df_vector_low_renamed.columns)
print(sub_df_vector_high_renamed.columns)
print(sub_df_graph_no_renamed.columns)
print(sub_df_graph_low_renamed.columns)
print(sub_df_graph_high_renamed.columns)

Index(['question', 'ground_truth', 'answer', 'contexts'], dtype='object')
Index(['question', 'ground_truth', 'answer', 'contexts'], dtype='object')
Index(['question', 'ground_truth', 'answer', 'contexts'], dtype='object')
Index(['question', 'ground_truth', 'answer', 'contexts'], dtype='object')
Index(['question', 'ground_truth', 'answer', 'contexts'], dtype='object')
Index(['question', 'ground_truth', 'answer', 'contexts'], dtype='object')


In [17]:
sub_df_vector_no_renamed['contexts'] = sub_df_vector_no_renamed['contexts'].apply(lambda x: [x])
ds_vector_no = Dataset.from_pandas(sub_df_vector_no_renamed)

sub_df_vector_low_renamed['contexts'] = sub_df_vector_low_renamed['contexts'].apply(lambda x: [x])
ds_vector_low = Dataset.from_pandas(sub_df_vector_low_renamed)

sub_df_vector_high_renamed['contexts'] = sub_df_vector_high_renamed['contexts'].apply(lambda x: [x])
ds_vector_high = Dataset.from_pandas(sub_df_vector_high_renamed)

sub_df_graph_no_renamed['contexts'] = sub_df_graph_no_renamed['contexts'].apply(lambda x: [x])
ds_graph_no = Dataset.from_pandas(sub_df_graph_no_renamed)

sub_df_graph_low_renamed['contexts'] = sub_df_graph_low_renamed['contexts'].apply(lambda x: [x])
ds_graph_low = Dataset.from_pandas(sub_df_graph_low_renamed)

sub_df_graph_high_renamed['contexts'] = sub_df_graph_high_renamed['contexts'].apply(lambda x: [x])
ds_graph_high = Dataset.from_pandas(sub_df_graph_high_renamed)

In [ ]:
ds_vector_no

In [20]:
llm = ChatOpenAI(model='gpt-4-turbo')

In [ ]:
res_vector_no = evaluate(
    ds_vector_no,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    llm=llm,
)

df_vector_no = res_vector_no.to_pandas()
# Save df_vector_no

In [ ]:
res_vector_low = evaluate(
    ds_vector_low,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    llm=llm,
)

df_vector_low = res_vector_low.to_pandas()
# Save df_vector_low

In [ ]:
res_vector_high = evaluate(
    ds_vector_high,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    llm=llm,
)

df_vector_high = res_vector_high.to_pandas()
# Save df_vector_high

In [ ]:
res_graph_no = evaluate(
    ds_graph_no,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    llm=llm,
)

df_graph_no = res_graph_no.to_pandas()
# Save df_graph_no

In [ ]:
res_graph_low = evaluate(
    ds_graph_low,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    llm=llm,
)

df_graph_low = res_graph_low.to_pandas()
# Save df_graph_low

In [ ]:
res_graph_high = evaluate(
    ds_graph_high,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
    llm=llm,
)

df_graph_high = res_graph_high.to_pandas()
# Save df_graph_high